In [51]:
from data.solver_config import SolverConfig
from data.stocks import Stock
from data.portfolios import Portfolio
from datetime import date
from Utils.portfolio_solver import PortfolioSolver, StockSnapshot
from cvxopt import matrix, solvers
import numpy as np
import pandas as pd

stocks: dict[Stock, StockSnapshot] = {
    Stock("AAPL", '1', "Apple"): StockSnapshot(0.2, {
        date(2014, 7, 1) : 14,
        date(2014, 7, 2) : 16,
        date(2014, 7, 3) : 12,
    }),
    Stock("MSFT", '2', "Microsoft"): StockSnapshot(0.8, {
        date(2014, 7, 1) : 99,
        date(2014, 7, 2) : 1,
        date(2014, 7, 3) : 250,
    })
}

config = SolverConfig()
solver = PortfolioSolver(stocks, config)


In [52]:
stock_list = list(solver.stocks.keys())
stock_count = len(stock_list)

# === Build price table from snapshots ===
price_data = pd.DataFrame({
    stock.ticker: snapshot.price_history
    for stock, snapshot in solver.stocks.items()
})

display(price_data)

,AAPL,MSFT
2014-07-01,14,99
2014-07-02,16,1
2014-07-03,12,250


In [53]:
# === Compute daily returns ===
var_returns = price_data.pct_change().dropna()
display(var_returns)

,AAPL,MSFT
2014-07-02,0.142857,-0.989899
2014-07-03,-0.250000,249.000000


In [54]:
# === Compute expected returns (mu) ===
mu: np.ndarray = var_returns.mean().values.reshape(-1, 1)
print(mu)

[[-5.35714286e-02]
 [ 1.24005051e+02]]


In [55]:
# === Compute covariance matrix (sigma) ===
sigma: np.ndarray = var_returns.cov().values
print(sigma)

[[ 7.71683673e-02 -4.91051587e+01]
 [-4.91051587e+01  3.12474748e+04]]


In [56]:
# === Normalize alpha scores ===
raw_scores = np.array([solver.stocks[stock].alpha_score for stock in stock_list])
normalized_scores = raw_scores / np.sum(raw_scores)
alpha_scores = normalized_scores.reshape(-1, 1)
print(alpha_scores)

[[0.2]
 [0.8]]


In [57]:
# === Optimization ===
p = matrix(solver.config.risk_aversion * sigma)
q = matrix(-mu - alpha_scores)

g = -np.eye(stock_count)
h = np.zeros(stock_count)

a = matrix(np.ones((1, stock_count)))
b = matrix(np.ones(1))

In [58]:
sol = solvers.qp(p, q, G=matrix(g), h=matrix(h), A=a, b=b)
weights = np.array(sol['x']).flatten()

     pcost       dcost       gap    pres   dres
 0: -1.1682e+00 -2.1827e+00  1e+00  1e-16  2e-02
 1: -1.1683e+00 -1.1784e+00  1e-02  2e-16  2e-04
 2: -1.1683e+00 -1.1684e+00  1e-04  2e-16  2e-06
 3: -1.1683e+00 -1.1683e+00  1e-06  1e-16  2e-08
Optimal solution found.


In [59]:
# === Build Portfolio ===
metadata = {
    stock: Portfolio.StockMetadata(weight, solver.stocks[stock].alpha_score)
    for stock, weight in zip(stock_list, weights)
}

df = pd.DataFrame({
    stock.ticker: {'Weight': md.weight, 'Alpha Score': md.alpha_score}
    for stock, md in metadata.items()
}).T  # Transpose for nicer layout

display(df)

,Weight,Alpha Score
AAPL,0.985175,0.2
MSFT,0.014825,0.8
